# Notebook
This is to test features of the image_translate library.

In [3]:
import os
import cv2
from PIL import Image, ImageDraw, ImageFont, ImageStat
import re
from openai import AzureOpenAI
import json
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt

# Import necessary functions from image_translate.py
from image_translate import retrieve_bounding_boxes_by_image_path, get_translated_text_data, FIXED_FONT_PATH_FOR_PLOTTING, get_text_color, create_filled_polygon_mask, draw_text_on_image, warp_image_to_bounding_box, get_average_color

def create_annotated_image(image_path, line_bounding_boxes, translated_text_list, plot=False):
    # Create output directory if it doesn't exist
    os.makedirs('./translated_images', exist_ok=True)
    image = Image.open(image_path).convert('RGBA')

    font_size = 40
    font = ImageFont.truetype(FIXED_FONT_PATH_FOR_PLOTTING, font_size)

    for line_info, translated_text in zip(line_bounding_boxes, translated_text_list):
        bounding_box = line_info['bounding_box']

        # Get the average color of the bounding box area
        bg_color = get_average_color(image, bounding_box)
        text_color = get_text_color(bg_color)

        # Create a mask to fill the bounding box area with the background color
        mask_image = create_filled_polygon_mask(
            bounding_box, image.size, bg_color)

        # Composite the mask onto the image to fill the bounding box
        image = Image.alpha_composite(image, mask_image)

        # Draw the translated text onto a temporary image
        text_image = draw_text_on_image(translated_text, font, text_color)

        # Convert the text image to an array and warp it to fit the bounding box
        text_image_array = np.array(text_image)
        warped_text_image = warp_image_to_bounding_box(
            text_image_array, bounding_box, image.width, image.height)

        # Convert the warped text image back to PIL format and paste it onto the original image
        warped_text_image_pil = Image.fromarray(warped_text_image)
        image = Image.alpha_composite(image, warped_text_image_pil)

    # Save the annotated image
    output_path = os.path.join(
        './translated_images', os.path.basename(image_path))
    image.save(output_path)

    # Display the image if needed
    if plot:
        plt.figure(figsize=(20, 10))
        plt.subplot(1, 2, 1)
        plt.imshow(image.convert('RGB'))
        plt.title("Annotated Image with Translated Text")
        plt.axis("off")

        original_image = Image.open(image_path)
        plt.subplot(1, 2, 2)
        plt.imshow(original_image)
        plt.title("Original Image")
        plt.axis("off")

        plt.show()
    return output_path

In [4]:
image_paths = ["./images/bicycle.png", "./images/road_sign.png"]
target_language = "Spanish"
image_path = image_paths[0]
lbb = retrieve_bounding_boxes_by_image_path(image_path)
text = get_translated_text_data(image_path, target_language)

Prompt: 
You are a translator that receives a batch of lines in an image . Given the following yaml file, please translate each line into Spanish. 
For each line, fill it in with the translation, respecting the context of the text.
Return only the yaml file, fully filled in.
- LIFE IS LIKE
- RIDING A BICYCLE
- TO
- KEEP YOUR BALANCE
- YOU MUST KEEP MOVING

Response: ['LA VIDA ES COMO', 'ANDAR EN BICICLETA', 'PARA', 'MANTENER EL EQUILIBRIO', 'DEBES SEGUIR MOVIÉNDOTE']


In [ ]:
create_annotated_image(image_path, lbb, text, plot=True)